This notebook demonstrates how to use `pretrained-microscopy-models` to perform binary segmentation on environmental barrier coating data. This is the EBC-1 dataset in the accompanying paper.

In [1]:
import os
import torch
import cv2
import random
import imageio

import numpy as np
import matplotlib.pyplot as plt
import pretrained_microscopy_models as pmm
import segmentation_models_pytorch as smp
import albumentations as albu

from pathlib import Path
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [2]:
# set random seeds for repeatability
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

## ML Model

In [3]:
# model parameters
architecture = 'UnetPlusPlus'
encoder = 'se_resnext50_32x4d'
pretrained_weights = 'micronet'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Create the UnetPlusPlus model with a se_resnext50_32x4d backbone 
# that is pre-trained on micronet
model = pmm.segmentation_training.create_segmentation_model(
    architecture=architecture,
    encoder = encoder,
    encoder_weights=pretrained_weights, # use encoder pre-trained on micronet
    classes=1 # Set 1 for binary classification (the background class is implicit)
    )

## Dataset

Note that the example annotations appear black because the pixel values are
0, 1, and 2 out of 255.

In [4]:
DATA_DIR = 'EBC1'

x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'train_annot')

x_valid_dir = os.path.join(DATA_DIR, 'val')
y_valid_dir = os.path.join(DATA_DIR, 'val_annot')

x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'test_annot')

#### Image Augmentation
Adding noise, contrast, random crops, flipping and other image augmentations can artifically augment the training dataset and make the model more robust to changes in these conditions

In [5]:
def get_training_augmentation(prob=0.3, power=1):
    """Need to make sure image shape is divisible by 32"""
    train_transform = [
        albu.HorizontalFlip(p=0.5), #only horizontal to preserve orientation
        albu.RandomCrop(512, 512),
        albu.GaussNoise(p=0.5),
        albu.RandomBrightnessContrast(p=0.3),

        albu.OneOf(
            [
                albu.Sharpen(p=1, alpha=(0.2, 0.5*power)),
                albu.Blur(blur_limit=3*power, p=1),
            ],
            p=0.3,
        ),
        
    ]

    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Need to make sure image shape is divisible by 32"""
    test_transform = [
        # instead of center cropping we will crop a little lower in the 
        # validation data to more accurately capture where the oxide layer is.
        albu.Crop(256, 256+100, 768, 768+100)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

### Create datasets

In [12]:
class Dataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations.
    Modified from https://github.com/qubvel/segmentation_models.pytorch
    
    
    Args:
        images (str or list): path to images folder or list of images
        masks (str): path to segmentation masks folder or list of images
        class_values (dict): values of classes to extract from segmentation mask. 
            Each dictionary value can be an integer or list that specifies the mask
            values that belong to the class specified by the corresponding dictionary key.
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    Note: If images and masks are directories the image and mask pairs should be 
    laballed "ImageName.tif" and "ImageNamemask.tif" respectively. Otherwise
    you should just pass the list of paths to images and masks.
    """
    
    def __init__(
            self, 
            images, 
            masks, 
            class_values,
            augmentation=None, 
            preprocessing=None,
    ):

        self.class_values = class_values
        self.augmentation = augmentation
        self.preprocessing = preprocessing

        # create list of image paths
        if type(images) is list:
            self.images_fps = images
            self.masks_fps = masks
        else:
            self.ids = os.listdir(images)
            self.images_fps = [os.path.join(images, image_id) for image_id in self.ids]
            # create list of annotation paths (MAY NEED TO ADJUST THIS LOGIC)
            self.masks_fps = [os.path.join(masks, image_id.replace('.tif', '_mask.tif')) for image_id in self.ids]
        
        
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        mask = cv2.imread(self.masks_fps[i], 1)
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # extract certain classes from mask (e.g. cars)
        masks = [np.all(mask == v, axis=-1) for v in self.class_values.values()]
        if len(masks) > 1:
            masks[0] = ~np.any(masks[1:], axis=0)
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask

    def __len__(self):
        return len(self.images_fps)

In [13]:
# how the images will be normalized. Use imagenet statistics even on micronet pre-training
preprocessing_fn = smp.encoders.get_preprocessing_fn(encoder, 'imagenet') 

# pixel values of the annotations for each mask.
# Note: for binary classification we only need to specify the forground class
# Note 2: you can specify multiple annotation values for the same class
#       (in this case the oxide cracks are also annotated, but not used in this example)
class_values = {'oxide': [1,2]}


# For multiclass segmentation we can pass the directory for images and masks.
# because they follow the naming convention "image1.tif" and "image1_mask.tif"
# in this case we need to pass the list of image and annotation paths explicitly
# because the annotation data does not follow that naming convention
train_images = [os.path.join(x_train_dir, image_id) for image_id in os.listdir(x_train_dir)]
train_masks = [os.path.join(y_train_dir, image_id) for image_id in os.listdir(y_train_dir)]

# train_images = [os.path.join(x_train_dir, image_id) for image_id in os.listdir(x_train_dir)]
# train_masks = [os.path.join(y_train_dir, image_id) for image_id in os.listdir(y_train_dir)]

# train_images = [os.path.join(x_train_dir, image_id) for image_id in os.listdir(x_train_dir)]
# train_masks = [os.path.join(y_train_dir, image_id) for image_id in os.listdir(y_train_dir)]

print(train_images)
print(train_masks)

training_dataset = pmm.io.Dataset(
    images=train_images,
    masks=train_masks,
    class_values=class_values,
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn)
)

validation_dataset = pmm.io.Dataset(
    images=x_valid_dir,
    masks=y_valid_dir,
    class_values=class_values,
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn)
)

test_dataset = pmm.io.Dataset(
    images=x_test_dir,
    masks=y_test_dir,
    class_values=class_values,
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn)
)

['EBC1\\train\\010417#4_S2480009.tif', 'EBC1\\train\\010417#4_S2480023.tif', 'EBC1\\train\\021519#3_1426C-100cyc0008.tif', 'EBC1\\train\\032919#10_100cyc0017.tif', 'EBC1\\train\\032919#6_100cyc0007.tif', 'EBC1\\train\\040617#3_S0410018.tif', 'EBC1\\train\\040617#3_S0410028.tif', 'EBC1\\train\\050317#3_S0440017.tif', 'EBC1\\train\\050317#3_S0440019.tif', 'EBC1\\train\\050317#3_S0440020.tif', 'EBC1\\train\\050317#3_S0440027.tif', 'EBC1\\train\\070516#1_S1160020.tif', 'EBC1\\train\\070516#3_S2470007.tif', 'EBC1\\train\\111418#4_1426_100Cyc0032.tif', 'EBC1\\train\\111418#4_1426_100Cyc0034.tif', 'EBC1\\train\\120718#4-B1_50cyc_14260025.tif', 'EBC1\\train\\122018#3_100cyc-1426C0017.tif', 'EBC1\\train\\S879_062216#3-31cyc0030.tif']
['EBC1\\train_annot\\010417#4_S2480009.tif', 'EBC1\\train_annot\\010417#4_S2480023.tif', 'EBC1\\train_annot\\021519#3_1426C-100cyc0008.tif', 'EBC1\\train_annot\\032919#10_100cyc0017.tif', 'EBC1\\train_annot\\032919#6_100cyc0007.tif', 'EBC1\\train_annot\\040617#3_S0

### Visualize datasets

In [14]:
### validation data

# we need to remove pre_processing to see the data
visualize_dataset = Dataset(
    images=train_images,
    masks=train_masks,
    class_values=class_values,
    augmentation=get_validation_augmentation(),
)

for im, mask in visualize_dataset:
    pmm.util.visualize(
        image=im,
        oxide_mask=mask.squeeze(),
    )


C:\Users\jstuckne\AppData\Local\Temp\3/ipykernel_10204/2183603038.py:55: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  masks = [np.all(mask == v, axis=-1) for v in self.class_values.values()]


ValueError: not enough values to unpack (expected 2, got 1)

In [11]:
for im, mask in training_dataset:
    pmm.util.visualize(
        image=im,
        oxide_mask=mask.squeeze(),
    )

In [18]:
im, mask = training_dataset

TypeError: 'Dataset' object is not an iterator